# BUS 774: Prediction Modeling Exercise

C Kaligotla

03-MAR-2025

## Problem Description

Given the scale of daily credit card transactions, detecting Credit Card fraud is challenging and is an ideal use case for ML applications.

We have some data about Credit Card fraud from some unnamed institute.

Data for training is available here: [Training Data](https://www.dropbox.com/scl/fi/og0ld7ao5d63ofdnkugtd/card_transdata_train.csv?rlkey=ss3wxcp85kwfq4pzgr8enf2if&dl=0)

The data has the following Features (Variables):
*   *distancefromhome*, numeric - the distance from home where the transaction happened.
* *distancefromlast_transaction*, numeric - the distance from last transaction happened.
* *ratiotomedianpurchaseprice*, numeric - Ratio of purchased price transaction to median purchase price.
* *repeat_retailer*, binary - Is the transaction happened from same retailer.
* *used_chip* , binary - Is the transaction through chip (credit card).
* *used_pin_number*, binary - Is the transaction happened by using PIN number.
* *online_order*, binary - Is the transaction an online order.

Target Variable:
* *fraud*, binary - Is the transaction fraudulent.

**Objective: ***Build an ML algorithm to predict fraudulent transactions.***

Data for testing your model performance is available here: [Test Data](https://www.dropbox.com/scl/fi/uk5xgx94oztqok2ef6w9o/card_transdata_test.csv?rlkey=8j7v1lo11dfenibaxxlxxkrf7&dl=1)

I asked the following questions:
1. What is the best metric to use for evaluating your ML model and why?
2. Build and train your model. Use the testing data to evaluate the model. Print out the confusion matrix from your  prediction model on the test data and report chosen metric
3. Identify your "best" model
4. I have new implementation data sets in class post-submission. Calculate your "best" model performance on this new implementation data and report your metric.
 Links:
 * [OOS_1_Data](https://www.dropbox.com/scl/fi/25tjo6wvwszygrq6kd750/card_OOS1.csv?rlkey=s1yfcjckludqahjf4bzn2u5no&dl=1)
 * [OOS_2_Data](https://www.dropbox.com/scl/fi/ntfp8rja3b2eofqk0g0ii/card_OOS2.csv?rlkey=0zqir93n37na2ac2o4zagfpd1&dl=1)
 * [OOS_3_Data](https://www.dropbox.com/scl/fi/o3g5k9aoe5rj6yfovac54/card_OOS3.csv?rlkey=rqail2dcnnc6qv2mv7sbdsnjp&dl=1)




## Preamble

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LassoCV
from sklearn.metrics import make_scorer,confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## Load Data and Explore

In [2]:
train_url = 'https://www.dropbox.com/scl/fi/og0ld7ao5d63ofdnkugtd/card_transdata_train.csv?rlkey=ss3wxcp85kwfq4pzgr8enf2if&e=1&dl=1'
test_url= 'https://www.dropbox.com/scl/fi/uk5xgx94oztqok2ef6w9o/card_transdata_test.csv?rlkey=8j7v1lo11dfenibaxxlxxkrf7&dl=1'

df=pd.read_csv(train_url)
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1,1,0,0,0
1,10.829943,0.175592,1.294219,1,0,0,0,0
2,5.091079,0.805153,0.427715,1,0,0,1,0
3,2.247564,5.600044,0.362663,1,1,0,1,0
4,44.190936,0.566486,2.222767,1,1,0,1,0


In [3]:
print(df.shape)
df.info()

(399999, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399999 entries, 0 to 399998
Data columns (total 8 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   distance_from_home              399999 non-null  float64
 1   distance_from_last_transaction  399999 non-null  float64
 2   ratio_to_median_purchase_price  399999 non-null  float64
 3   repeat_retailer                 399999 non-null  int64  
 4   used_chip                       399999 non-null  int64  
 5   used_pin_number                 399999 non-null  int64  
 6   online_order                    399999 non-null  int64  
 7   fraud                           399999 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 24.4 MB


Note: Everything here is numerical, so we don't have to convert anything.

If for instance, fraud was "Y" and "N" instead of 1 and 0, we'd use code like this:

```
y = data['fraud'].apply(lambda x: 1 if x == 'Y' else 0)  # Encode target
```

There's no missing data, so we don't have to worry about imputation.

In [4]:
df.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,399999.000000,399999.000000,399999.000000,399999.000000,399999.000000,399999.000000,399999.000000,399999.000000
mean,26.736294,4.982687,1.819569,0.881702,0.350601,0.101473,0.651657,0.087148
std,66.599017,22.296888,2.794580,0.322961,0.477159,0.301954,0.476446,0.282052
min,0.004874,0.000419,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.872526,0.296428,0.474748,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.975605,0.998808,0.997292,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.779398,3.338713,2.095440,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,2663.513077,266.689692,1.000000,1.000000,1.000000,1.000000,1.000000


### # Separate features and target

In [5]:
X = df.drop('fraud', axis=1)
y = df['fraud']
print(X.head())
print(X.shape)
print(y.head())
print(y.shape)

   distance_from_home  distance_from_last_transaction  \
0           57.877857                        0.311140   
1           10.829943                        0.175592   
2            5.091079                        0.805153   
3            2.247564                        5.600044   
4           44.190936                        0.566486   

   ratio_to_median_purchase_price  repeat_retailer  used_chip  \
0                        1.945940                1          1   
1                        1.294219                1          0   
2                        0.427715                1          0   
3                        0.362663                1          1   
4                        2.222767                1          1   

   used_pin_number  online_order  
0                0             0  
1                0             0  
2                0             1  
3                0             1  
4                0             1  
(399999, 7)
0    0
1    0
2    0
3    0
4    0
Name: fra

Pre-Process the data - Transform (standardization/normalization) and convert to array)

See: https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe/

In [6]:
X = StandardScaler().fit_transform(X)
X[:5] # first 5 rows

array([[ 0.46759852, -0.2095159 ,  0.04522012,  0.36629194,  1.3609725 ,
        -0.33605402, -1.36774657],
       [-0.23883793, -0.21559517, -0.18798919,  0.36629194, -0.7347687 ,
        -0.33605402, -1.36774657],
       [-0.32500842, -0.18735975, -0.49805562,  0.36629194, -0.7347687 ,
        -0.33605402,  0.7311296 ],
       [-0.36770453,  0.02768806, -0.52133356,  0.36629194,  1.3609725 ,
        -0.33605402,  0.7311296 ],
       [ 0.26208591, -0.19806379,  0.14427891,  0.36629194,  1.3609725 ,
        -0.33605402,  0.7311296 ]])

In [7]:
# convert to numpy array - since it's just 0 and 1, i'm not applying a transform
y= y.to_numpy()
y[:100] #first 100 obs

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Test-train Split for internal test data (validation data) to choose our *best* model

In [8]:
# I'm creating a separate internal test / validation set to choose best model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(319999, 7)
(319999,)
(80000, 7)
(80000,)


## Build Models

Let's build the following models / classifiers to choose the **Best**
1. Logistic Regression
2. LASSO Logistic Regression
3. Ridge Logistic Regression
4. Decision Tree / CART
5. Random Forest
6. KNN / K-Nearest Neighbors

We'll use 5-fold CV to train/fit our models using

```
# Using StratifiedKFold to maintain the same proportion of classes in each fold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
```

**First STEP - WHAT is the RIGHT METRIC to use? and why?**

Using f1 here as default.

Also showing you prompts to generate code


***General Workflow:***
1. Define classifier (model)
2. Define the StratifiedKFold cross-validator (need to do it once)
3. Train model using CV on training data, report evaluations across folds)
4. Fit model on entire train data and predict on internal test (validation) data
5. Report Confusion Matrix and F1 Score

In [9]:
f1_scores_training = {} # to store scores


### 1. Logistic Regression Model

In [10]:
# prompt: Build a logistic regression classifier on X_train and y_train and using  5-fold cv using kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42).

# Define the logistic regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Define the StratifiedKFold cross-validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(log_reg, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))


Cross-validation scores: [0.72845981 0.72096757 0.72611869 0.72664732 0.72337676]
Mean cross-validation score: 0.7251140305103352


In [11]:
# prompt: Use the best log_reg model on y_test and y_train. Report confusion matrix and f1 score

# Fit the best model (log_reg in this case) on the entire training data
log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = log_reg.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))

f1_scores_training['log_reg'] = f1_score(y_test, y_pred)

[[72440   500]
 [ 2789  4271]]
F1 Score: 0.7220015214267602


### 2. LASSO Logistic Regression


In [12]:
# prompt: Repeat the above but using LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)

from sklearn.metrics import confusion_matrix

# Define the LASSO logistic regression model
lasso_log_reg = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)

# Define the StratifiedKFold cross-validator
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(lasso_log_reg, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))

# Fit the model on the entire training data
lasso_log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = lasso_log_reg.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))

f1_scores_training['lasso_log_reg'] = f1_score(y_test, y_pred)

Cross-validation scores: [0.72865397 0.72124131 0.72650386 0.72656922 0.72323928]
Mean cross-validation score: 0.7252415272736824
[[72438   502]
 [ 2788  4272]]
F1 Score: 0.7219874936623288


### 3. Ridge Logistic Regression


In [13]:
# prompt: repeat for Ridge Regression

# Define the Ridge logistic regression model
ridge_log_reg = RidgeClassifier(random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(ridge_log_reg, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))

# Fit the model on the entire training data
ridge_log_reg.fit(X_train, y_train)

# Predict on the test set
y_pred = ridge_log_reg.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))

f1_scores_training['ridge_log_reg'] = f1_score(y_test, y_pred)

Cross-validation scores: [0.26210153 0.27535808 0.26179693 0.26538745 0.27115666]
Mean cross-validation score: 0.2671601329977647
[[72529   411]
 [ 5903  1157]]
F1 Score: 0.26819656930922575


### 4. Decision Tree / CART


In [14]:
# prompt: Repeat code for DecisionTree/CART using DecisionTreeClassifier(criterion='entropy', max_depth = 10, random_state=42)

# Define the Decision Tree Classifier model
CART = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(CART, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))

# Fit the model on the entire training data
CART.fit(X_train, y_train)

# Predict on the test set
y_pred = CART.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))

f1_scores_training['CART'] = f1_score(y_test, y_pred)

Cross-validation scores: [0.99991008 0.99991006 0.99982014 0.99991006 0.99955008]
Mean cross-validation score: 0.9998200856219979
[[72940     0]
 [    1  7059]]
F1 Score: 0.9999291734542106


### 5. Random Forest

In [15]:
# prompt: Repeat using RandomForestClassifier(max_depth = 10, random_state=42)

# Define the Random Forest Classifier model
RF = RandomForestClassifier(max_depth=10, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(RF, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))

# Fit the model on the entire training data
RF.fit(X_train, y_train)

# Predict on the test set
y_pred = RF.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))

f1_scores_training['RandomForest'] = f1_score(y_test, y_pred)

Cross-validation scores: [1.         0.99973014 0.99982011 0.99991006 0.99955008]
Mean cross-validation score: 0.9998020789793417
[[72940     0]
 [    2  7058]]
F1 Score: 0.9998583368749114


### 6. KNN / K-Nearest Neighbors


In [16]:
# prompt: Repeat for KNeighborsClassifier(n_neighbors=5)

# Define the KNN Classifier model
knn = KNeighborsClassifier(n_neighbors=5)

# Perform cross-validation
cv_scores = cross_val_score(knn, X_train, y_train, cv=kf, scoring='f1')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", np.mean(cv_scores))

# Fit the model on the entire training data
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the F1 score
print("F1 Score:", f1_score(y_test, y_pred))
f1_scores_training['knn'] = f1_score(y_test, y_pred)

Cross-validation scores: [0.98809309 0.98789302 0.98889391 0.98924731 0.98869494]
Mean cross-validation score: 0.988564455343996
[[72905    35]
 [   80  6980]]
F1 Score: 0.991829484902309


In [17]:
f1_scores_training

{'log_reg': 0.7220015214267602,
 'lasso_log_reg': 0.7219874936623288,
 'ridge_log_reg': 0.26819656930922575,
 'CART': 0.9999291734542106,
 'RandomForest': 0.9998583368749114,
 'knn': 0.991829484902309}

In [19]:
# prompt: What is the best model from the 6?

# Based on the provided code, the best model is the one with the highest F1 score
# on the internal test set.  You would need to manually compare the F1 scores
# printed for each model to determine the best one.  The code does not store
# these values in a way that allows for automated comparison.

# Example of how to compare manually:
# Look for the "F1 Score:" lines at the end of each model section.
# The model with the highest F1 score is considered the best in this scenario.

#To compare and find the best model:

best_model = max(f1_scores_training, key=f1_scores_training.get)
print(f"Best model based on F1-score is: {best_model} with F1-score {f1_scores_training[best_model]}")


Best model based on F1-score is: CART with F1-score 0.9999291734542106


## Evaluate Models on Provided Test Data

### Transform TEST data as needed

In [20]:
df2 = pd.read_csv(test_url)

In [21]:
X_TEST = df2.drop('fraud', axis=1)
y_TEST = df2['fraud']
print(X_TEST.shape)
print(y_TEST.shape)

(100000, 7)
(100000,)


In [22]:
X_TEST = StandardScaler().fit_transform(X_TEST)
y_TEST= y_TEST.to_numpy()

### EVALUATE MODELS ON the provided X_TEST and y_TEST -- models have not been trained on this!

In [32]:
# prompt: Evaluate all models above on X_TEST and y_TEST. Report f1 scores in a neat table.

# Create a list to store the model names and their corresponding f1 scores
f1_scores_TEST = []

# Evaluate each model on X_TEST and y_TEST and store the f1 scores
models = [log_reg, lasso_log_reg, ridge_log_reg, CART, RF, knn]
model_names = ['log_reg', 'lasso_log_reg', 'ridge_log_reg', 'CART', 'RandomForest', 'knn'] # Create a list of model names

for model in models:
    y_pred = model.predict(X_TEST)
    f1 = f1_score(y_TEST, y_pred)
    f1_scores_TEST.append(f1)

In [37]:
# Create a DataFrame to display the results in a table
results_df = pd.DataFrame({"Model": model_names, "F1 Score TEST DATA": f1_scores_TEST}) # Use model_names here

results_df['f1_scores_InternalTesting'] = results_df['Model'].map(f1_scores_training) # Map using 'Model' column
# Display the table
results_df


,Model,F1 Score TEST DATA,f1_scores_InternalTesting
0,log_reg,0.702917,0.722002
1,lasso_log_reg,0.703179,0.721987
2,ridge_log_reg,0.249364,0.268197
3,CART,0.946805,0.999929
4,RandomForest,0.953962,0.999858
5,knn,0.951592,0.991829


## Choose "Best Model" and Fine-Tune if needed

Assume RandomForest Wins.
We can fine-tune (play around hyperparameters to see if we can improve), or at least, train the model on full data

In [40]:
# prompt: combine X and X_TEST

# Combine X and X_TEST
combined_X = np.concatenate((X, X_TEST), axis=0)
combined_y = np.concatenate((y, y_TEST), axis=0)

print(combined_X.shape)
combined_y.shape


(499999, 7)


(499999,)

In [41]:
# Retrain the Random Forest model on the entire dataset
RF_final = RandomForestClassifier(max_depth=10, random_state=42)
RF_final.fit(X, y)


RandomForestClassifier(max_depth=10, random_state=42)

## Evaluate "Best Model" on provided Implementation/OOS DataSets

In [42]:
oos1_url = 'https://www.dropbox.com/scl/fi/25tjo6wvwszygrq6kd750/card_OOS1.csv?rlkey=s1yfcjckludqahjf4bzn2u5no&dl=1'
oos2_url = 'https://www.dropbox.com/scl/fi/ntfp8rja3b2eofqk0g0ii/card_OOS2.csv?rlkey=0zqir93n37na2ac2o4zagfpd1&dl=1'
oos3_url = 'https://www.dropbox.com/scl/fi/o3g5k9aoe5rj6yfovac54/card_OOS3.csv?rlkey=rqail2dcnnc6qv2mv7sbdsnjp&dl=1'

In [43]:
oos1 = pd.read_csv(oos1_url)
oos2 = pd.read_csv(oos2_url)
oos3 = pd.read_csv(oos3_url)

In [44]:
print(oos1.shape)
print(oos2.shape)
print(oos3.shape)

(500, 8)
(1000, 8)
(499, 8)


In [46]:
#Apply scalar transforms to all OOS like before

X_oos1 = oos1.drop('fraud', axis=1)
y_oos1 = oos1['fraud']
X_oos1 = StandardScaler().fit_transform(X_oos1)
y_oos1 = y_oos1.to_numpy()

X_oos2 = oos2.drop('fraud', axis=1)
y_oos2 = oos2['fraud']
X_oos2 = StandardScaler().fit_transform(X_oos2)
y_oos2 = y_oos2.to_numpy()

X_oos3 = oos3.drop('fraud', axis=1)
y_oos3 = oos3['fraud']
X_oos3 = StandardScaler().fit_transform(X_oos3)
y_oos3 = y_oos3.to_numpy()

### EVALUATION TIME!

In [47]:
# prompt: Run RF_final.fit on X_oos1 and y_oos1. Report confusion matrix and f1 score

y_pred_oos1 = RF_final.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score:", f1_score(y_oos1, y_pred_oos1))


[[235  15]
 [189  61]]
F1 Score: 0.37423312883435583


In [48]:
y_pred_oos2 = RF_final.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score:", f1_score(y_oos2, y_pred_oos2))

[[440  60]
 [375 125]]
F1 Score: 0.36496350364963503


In [49]:
y_pred_oos3 = RF_final.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score:", f1_score(y_oos3, y_pred_oos3))


[[341   0]
 [150   8]]
F1 Score: 0.0963855421686747


WTH!



---

Maybe another model was right. Maybe the RF without re-training on all data


---



In [54]:
y_pred_oos1 = RF.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = RF.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = RF.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[235  15]
 [189  61]]
F1 Score: OOS1 0.37423312883435583
[[440  60]
 [375 125]]
F1 Score: OOS2 0.36496350364963503
[[341   0]
 [150   8]]
F1 Score: OOS3 0.0963855421686747


In [55]:
y_pred_oos1 = CART.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = CART.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = CART.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[235  15]
 [189  61]]
F1 Score: OOS1 0.37423312883435583
[[440  60]
 [375 125]]
F1 Score: OOS2 0.36496350364963503
[[341   0]
 [150   8]]
F1 Score: OOS3 0.0963855421686747


In [56]:
y_pred_oos1 = knn.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = knn.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = knn.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[235  15]
 [190  60]]
F1 Score: OOS1 0.36923076923076925
[[437  63]
 [377 123]]
F1 Score: OOS2 0.358600583090379
[[341   0]
 [149   9]]
F1 Score: OOS3 0.10778443113772455


In [58]:
y_pred_oos1 = ridge_log_reg.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = ridge_log_reg.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = ridge_log_reg.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[250   0]
 [238  12]]
F1 Score: OOS1 0.0916030534351145
[[495   5]
 [480  20]]
F1 Score: OOS2 0.0761904761904762
[[337   4]
 [150   8]]
F1 Score: OOS3 0.09411764705882353


In [60]:
y_pred_oos1 = lasso_log_reg.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = lasso_log_reg.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = lasso_log_reg.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[250   0]
 [189  61]]
F1 Score: OOS1 0.39228295819935693
[[496   4]
 [390 110]]
F1 Score: OOS2 0.3583061889250814
[[333   8]
 [150   8]]
F1 Score: OOS3 0.09195402298850575


In [61]:
y_pred_oos1 = log_reg.predict(X_oos1)
print(confusion_matrix(y_oos1, y_pred_oos1))
print("F1 Score: OOS1", f1_score(y_oos1, y_pred_oos1))
y_pred_oos2 = log_reg.predict(X_oos2)
print(confusion_matrix(y_oos2, y_pred_oos2))
print("F1 Score: OOS2", f1_score(y_oos2, y_pred_oos2))
y_pred_oos3 = log_reg.predict(X_oos3)
print(confusion_matrix(y_oos3, y_pred_oos3))
print("F1 Score: OOS3", f1_score(y_oos3, y_pred_oos3))

[[250   0]
 [189  61]]
F1 Score: OOS1 0.39228295819935693
[[496   4]
 [390 110]]
F1 Score: OOS2 0.3583061889250814
[[333   8]
 [150   8]]
F1 Score: OOS3 0.09195402298850575




---

***DO AT HOME: Now make a Neural Network on this data -- How does that perform and compare?***

---

